In [17]:
import pandas as pd
import numpy as np
import os
import csv
import yaml
import datetime

In [18]:
homedir = os.path.expanduser("~")
cricket = "Documents\\Data\\cricketbbb\\"
folders = os.listdir(os.path.join(homedir, cricket))
yaml_folders= [os.path.join(homedir, cricket, n) for n in folders if "yaml" in n]

In [34]:
subinfo = ["venue", "date", "dates", "gender", "match_type"]
legit_check = 50
values = {"extras_noballs": 0, "extras_wides": 0, "extras_byes":0, "extras_legbyes": 0}
for fold in yaml_folders:
    typ = fold.split("\\")[-1].split("_")[0]
    dest = os.path.join(homedir, cricket, typ+"_csv\\")
    try:
        os.mkdir(dest)
    except:
        print("folder already exists")

    for yam in os.listdir(fold):
        if yam != "README.txt":
            with open(os.path.join(fold, yam), 'r') as stream:
                try:
                    match_id = yam.split(".")[0]
                    if match_id not in [n.split(".")[0] for n in os.listdir(dest)]:
                        csvlist = []
                        test = (yaml.safe_load(stream))
                        test_info = test["info"]
                        test_innings = test["innings"]
                        for inn in test_innings:
                            for key in inn.keys():
                                batting_team = inn[key]["team"]
                                for ball in inn[key]["deliveries"]:
                                    csvdict = {"batting_team": batting_team, "inning": key[0], "match_id": match_id}
                                    csvdict["overball"] = list(ball.keys())[0]
                                    csvdict["over"], csvdict["ball"] = str(csvdict["overball"]).split(".")
                                    subdict = ball[csvdict["overball"]]
                                    for subkey1 in subdict.keys():
                                        if isinstance(subdict[subkey1], dict):
                                            for subkey2 in subdict[subkey1].keys():
                                                csvdict["_".join([subkey1, subkey2])] = subdict[subkey1][subkey2]
                                        else:
                                            csvdict[subkey1] = subdict[subkey1]
                                    csvlist.append(csvdict)
                        df = pd.DataFrame(csvlist)
                        info_keys = [x for x in test_info.keys() if x in subinfo]
                        df_info = pd.DataFrame([[test_info[x] for x in info_keys]] * len(df), columns = info_keys)
                        new_df = pd.concat([df, df_info], axis = 1, sort = True)
                        if "dates" in new_df.columns:
                            new_df["date"] = new_df["dates"].apply(lambda x: x[0])

                        new_df["date"] = pd.to_datetime(new_df["date"])
                        new_df["year"] = new_df["date"].apply(lambda x: x.year)
                        new_df["month"] = new_df["date"].apply(lambda x: x.month)
                        new_df["day"] = new_df["date"].apply(lambda x: x.day)
                        

                        new_df.fillna(value = values, inplace = True)
                        for extra in values.keys():
                            if extra not in new_df.columns:
                                new_df[extra] = 0
                        
                        new_df["ball_legit"] = (legit_check - (new_df["extras_noballs"] + new_df["extras_wides"]))//legit_check

                        new_df.to_csv(dest + str(match_id) + ".csv")


                except yaml.YAMLError as exc:
                    print(exc)

folder already exists


In [78]:
bowling_agg = {"ball_legit": np.sum, "runs_batsman": np.sum, "extras_noballs": np.sum, "extras_wides": np.sum}
batting_agg = {"ball_legit": np.sum, "runs_batsman": np.sum}

batting = []
bowling = []
for test in os.listdir(dest):
    df = pd.read_csv(dest + test)
    try:
        a01 = df.groupby(["inning", "batting_team", "batsman"])
        a02 = df.groupby(["inning", "batting_team", "wicket_player_out"])
        a1 = a01.agg(batting_agg)
        a2 = a02.runs_batsman.count()
        a = pd.concat([a1, a2], axis = 1, sort = True).fillna(0)
        a.columns = ["balls", "runs", "out"]
        batting.append(a)
        
        b0 = df.groupby(["inning", "bowler"])
        b = b0.agg(bowling_agg)
        b2 = b0.wicket_kind.value_counts().unstack().fillna(0)
        b["wickets"] = sum([b2[x] for x in b2.columns if x != "run out"])
        b.fillna(0, inplace = True)
        bowling.append(b)
    except:
        print("??")
    
A = pd.concat(batting).reset_index()
B = pd.concat(bowling).reset_index()

??


In [103]:
agg = {"balls": [np.sum, "count", np.mean, np.median, np.max], "runs": [np.sum, np.median, np.max], "out":np.sum }

m = 0
summary = A.groupby(["level_1", "level_2"]).agg(agg)
summary.columns = ["_".join(n) for n in summary.columns]
summary.loc[summary.out_sum>m, "average"] = summary.loc[summary.out_sum>m, "runs_sum"] / summary.loc[summary.out_sum>m, "out_sum"]
summary["strike_rate"] =  100 * summary["runs_sum"] / summary["balls_sum"]
summary.loc[summary.balls_count > 20, :].sort_values("average", ascending = False)

balls_sum  balls_count  balls_mean  \
level_1      level_2                                                 
Australia    SPD Smith             9771.0          103   94.864078   
             AC Voges              2660.0           31   85.806452   
West Indies  S Chanderpaul        10001.0           97  103.103093   
South Africa AB de Villiers       11034.0          116   95.120690   
Sri Lanka    KC Sangakkara        11825.0          115  102.826087   
Pakistan     Younis Khan          10385.0          108   96.157407   
Sri Lanka    TT Samaraweera        6208.0           70   88.685714   
South Africa HM Amla              13508.0          143   94.461538   
England      JE Root               9027.0          108   83.583333   
India        SR Tendulkar          8285.0           98   84.540816   
Australia    SM Katich             5882.0           61   96.426230   
India        CA Pujara             7975.0           83   96.084337   
South Africa JH Kallis             7691.0           89   86.415730   
             Q de Kock             1934.0           32   60.437500   
New Zealand  KS Williamson        10168.0          113   89.982301   
South Africa GC Smith              7785.0           99   78.636364   
India        VVS Laxman            6729.0           83   81.072289   
Australia    MJ Clarke            12134.0          153   79.307190   
India        V Sehwag              4906.0           93   52.752688   
             V Kohli               8159.0           99   82.414141   
New Zealand  LRPL Taylor          10077.0          140   71.978571   
West Indies  CH Gayle              3361.0           56   60.017857   
India        AM Rahane             5266.0           66   79.787879   
West Indies  RR Sarwan             2830.0           32   88.437500   
Pakistan     Misbah-ul-Haq        10013.0          114   87.833333   
South Africa ND McKenzie           2853.0           29   98.379310   
Bangladesh   Mominul Haque         3083.0           40   77.075000   
England      AN Cook              19416.0          209   92.899522   
Zimbabwe     BRM Taylor            2000.0           26   76.923077   
Pakistan     Azhar Ali            11893.0          116  102.525862   
...                                   ...          ...         ...   
Australia    NM Lyon               1560.0           85   18.352941   
West Indies  JE Taylor              802.0           47   17.063830   
Bangladesh   Shahadat Hossain       813.0           44   18.477273   
India        Z Khan                1145.0           58   19.741379   
South Africa M Morkel              1549.0           85   18.223529   
Sri Lanka    RAS Lakmal             967.0           58   16.672414   
England      ST Finn                899.0           44   20.431818   
Pakistan     Umar Gul               845.0           47   17.978723   
             Saeed Ajmal           1080.0           53   20.377358   
West Indies  KAJ Roach             1557.0           61   25.524590   
Bangladesh   Taijul Islam           413.0           21   19.666667   
West Indies  FH Edwards             691.0           38   18.184211   
England      JM Anderson           2248.0          140   16.057143   
South Africa M Ntini                286.0           23   12.434783   
Bangladesh   Rubel Hossain          481.0           38   12.657895   
South Africa Imran Tahir            234.0           21   11.142857   
Sri Lanka    UWMBCA Welegedara      419.0           30   13.966667   
India        PP Ojha                488.0           27   18.074074   
New Zealand  IE O'Brien             488.0           24   20.333333   
India        I Sharma              2005.0          105   19.095238   
             UT Yadav               465.0           35   13.285714   
Pakistan     Wahab Riaz             671.0           38   17.657895   
             Rahat Ali              372.0           31   12.000000   
             Junaid Khan            305.0           28   10.892857   
India        S Sreesanth            279.0 

In [ ]:
agg = {"balls": [np.sum, "count", np.mean, np.median, np.max], "runs": [np.sum, np.median, np.max], "out":np.sum }

m = 0
summary = B.groupby(["level_1", "level_2"]).agg(agg)
summary.columns = ["_".join(n) for n in summary.columns]
summary.loc[summary.out_sum>m, "average"] = summary.loc[summary.out_sum>m, "runs_sum"] / summary.loc[summary.out_sum>m, "out_sum"]
summary["strike_rate"] =  100 * summary["runs_sum"] / summary["balls_sum"]
summary.loc[summary.balls_count > 20, :].sort_values("average", ascending = False)

In [107]:
B.groupby("bowler").sum().sort_values("wickets", ascending = False)

,inning,ball_legit,runs_batsman,extras_noballs,extras_wides,wickets
bowler,,,,,,
JM Anderson,458,23123.0,10846,31.0,63.0,409.0
SCJ Broad,466,21726.0,10757,71.0,81.0,390.0
DW Steyn,290,14289.0,7257,33.0,94.0,341.0
HMRKB Herath,303,19869.0,9064,6.0,7.0,337.0
MG Johnson,345,15342.0,8381,89.0,135.0,305.0
R Ashwin,217,14439.0,6931,0.0,6.0,275.0
NM Lyon,337,16460.0,8693,2.0,1.0,260.0
M Morkel,316,14311.0,7116,241.0,103.0,258.0
GP Swann,255,15349.0,7637,0.0,5.0,255.0
